In [1]:
# Import section
import cv2
import numpy as np
from os import scandir, getcwd, rename
import requests
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from PIL import Image
import time
import json
import re
import pandas as pd

In [2]:
# Rutas parámetros de archivos de input y lista d files
ruta = "D:\\GyS\\proyectos\\demoAzureVision\\cupones\\"
ruta = "C:\\git\\cuponesWong\\CuponesWong\\notebooks_flow\\input\\"
lista = [arch.name for arch in scandir(ruta) if arch.is_file()]

# URL variables
vision_base_url = "https://eastus2.api.cognitive.microsoft.com/vision/v2.0/"
# vision_base_url = "https://southcentralus.api.cognitive.microsoft.com/vision/v2.0/"
ocr_url = vision_base_url + "recognizeText?mode=Handwritten"
subscription_key = "db6ddba1087844a58b0329c27b44ff5e"
# subscription_key = "e3d29d8cac4748328976033879af3216"

dic = {}
files = []
jsons = []

i = 1
files_count = 0

# Recorriendo la lista de archivos en la ruta
for file in lista:
    if files_count>=19:
        print('Sleeping 65 secs')
        time.sleep(65)
        files_count=0
    
    # ruta por archivo
    path = ruta + file
    print("Reading {}".format(path))
    image = open(path,"rb")
    d = image.read()
    headers = {'Content-Type': 'application/octet-stream', 'Ocp-Apim-Subscription-Key': subscription_key}
    # data    = {'url': image_url}
    response = requests.post(ocr_url, headers=headers, data=d)
    
    # Holds the URI used to retrieve the recognized text.
    operation_url = response.headers["Operation-Location"]
    # print("Url to get text analized response:{}".format(operation_url))

    # The recognized text isn't immediately available, so poll to wait for completion.
    analysis = {}
    poll = True
    
    while (poll):
        response_final = requests.get(
            operation_url, headers=headers)
        analysis = response_final.json()
        # print(analysis)
        time.sleep(5)
        if ('recognitionResult' in analysis):
            poll= False 
        if ("status" in analysis and analysis['status'] == 'Failed'):
            poll= False
    
    obj = json.loads(response_final.text)
    dic[file] = obj
    
    files.append(path)
    jsons.append(obj)
    image.close()
    files_count +=1
print("Terminó : D")

Reading C:\git\cuponesWong\CuponesWong\notebooks_flow\input\MarzoCompras_0202072019164448_00043.jpg
Reading C:\git\cuponesWong\CuponesWong\notebooks_flow\input\MarzoCompras_0202072019164503_00537.jpg
Reading C:\git\cuponesWong\CuponesWong\notebooks_flow\input\MarzoCompras_0202072019164503_00546.jpg
Reading C:\git\cuponesWong\CuponesWong\notebooks_flow\input\MarzoCompras_0202072019164511_00837.jpg
Terminó : D


In [ ]:
azure_files = np.array(files)
azure_jsons = np.array(jsons)

In [ ]:
'''
# Saving checked files
ruta = 'D:\\GyS\\proyectos\\demoAzureVision\\numpys_muestra\\'
np.save(ruta+'muestra_azure_objs', azure_jsons)
np.save(ruta+'muestra_files', azure_files)
'''

In [2]:
# Loading muestra numpy arrays
ruta = 'D:\\GyS\\proyectos\\demoAzureVision\\numpys_muestra\\'
azure_files = np.load(ruta+'muestra_files.npy',allow_pickle=True)
azure_jsons = np.load(ruta+'muestra_azure_objs.npy',allow_pickle=True)

In [3]:
azure_files.size, azure_jsons.size

(1059, 1059)

In [5]:
def get_mail_from_str(azure_line):
    a = azure_line.find('Email:')
    b = azure_line.find('CentroComercial:')
    
    if a!=-1 and b!=-1:
        return azure_line[a+len('Email:'):b]
    else:
        return ""
    # print('Mail encontrado en {}'.format(a))
    # print('Centro come encontrado en {}'.format(b))
    # print('R : {}'.format(li[a+len('Email:'):b]))

In [6]:
azure_files[47:52]

array(['D:\\GyS\\proyectos\\demoAzureVision\\cupones\\MarzoCompras_0203072019122714_00118.jpg',
       'D:\\GyS\\proyectos\\demoAzureVision\\cupones\\MarzoCompras_0203072019122717_00221.jpg',
       'D:\\GyS\\proyectos\\demoAzureVision\\cupones\\MarzoCompras_0203072019122717_00226.jpg',
       'D:\\GyS\\proyectos\\demoAzureVision\\cupones\\MarzoCompras_0203072019122720_00366.jpg',
       'D:\\GyS\\proyectos\\demoAzureVision\\cupones\\MarzoCompras_0203072019122722_00423.jpg'],
      dtype='<U84')

In [7]:
# for i in range(azure_jsons.size):
for i in range(0,1):
    # Getting json info
    j=0
    # Reading the image
    print('Para: {}'.format(azure_files[i]))
    image = cv2.imread(azure_files[i])
    
    for line in azure_jsons[i]['recognitionResult']['lines']:
        # final_str += line['text']
        j+=1
        print('Line {}: {},{}'.format(j,line['text'],line['boundingBox']))
        cv2.rectangle(image,(line['boundingBox'][0],line['boundingBox'][1]),(line['boundingBox'][2],line['boundingBox'][5]),(0,0,0),1)
    
    cv2.imshow('image',image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Para: D:\GyS\proyectos\demoAzureVision\cupones\MarzoCompras-0303072019121014_00343.jpg
Line 1: Nombres y Apellidos: TERESA HURTADO,[33, 37, 829, 17, 831, 73, 35, 94]
Line 2: CASTILLO,[198, 82, 456, 81, 457, 136, 199, 137]
Line 3: DNI:,[36, 166, 88, 165, 89, 193, 37, 194]
Line 4: of yoly / Telefono: 536 765.,[171, 133, 817, 135, 816, 196, 170, 195]
Line 5: Direccion:,[34, 222, 164, 223, 163, 253, 34, 253]
Line 6: COMA,[168, 208, 455, 204, 456, 253, 169, 258]
Line 7: AU . 22 DE AGOSTO) 265,[80, 263, 810, 251, 810, 306, 81, 319]
Line 8: Distrito: Tere hurtado of inthe,[36, 323, 834, 307, 836, 369, 37, 386]
Line 9: E-mail:,[35, 396, 122, 396, 122, 425, 35, 424]
Line 10: Centro Comercial:,[36, 454, 265, 455, 264, 486, 35, 485]
Line 11: - COMPS,[554, 446, 810, 423, 814, 469, 558, 493]


In [9]:
image.shape

NameError: name 'image' is not defined

In [8]:
dn_eqs = {'%':'90',')':'',',':'','$':'5','f':'7','&':'6','+':'7','g':'9','l':'1','o':'0','y':'4','/':'1','.':'','-':''}
cels = []
dnis = []
mails = []
lines = []

# for key in dic.keys():
for i in range(azure_jsons.size):
    final_str = ""
    # print("{}".format(len(dic[key]['recognitionResult']['lines'])))
    for line in azure_jsons[i]['recognitionResult']['lines']:
        final_str += line['text']
        # print("{},{},{}".format(key,line['text'],line['boundingBox']))
    lines.append(final_str)
    
    # print("{}".format(final_str))
    final_str = final_str.replace(' ','')
    for k in dn_eqs.keys():
        final_str = final_str.replace(k,dn_eqs[k])
    # print("{}".format(final_str))
    
    mails.append(get_mail_from_str(final_str))
    
    match_dni = re.search(r"[0-9]{8}", final_str)
    match_cel = re.search(r"[0-9]{9}", final_str)
    
    # print("{}\n{}".format(match_dni, match_cel))
    i=0
    while i<1:
        try:
            # print("dni={},start={},end={}".format(match_dni.group(i), match_dni.start(), match_dni.end()))
            dni=match_dni.group(i)
        except:
            dni = ""
        try:
            # print("cel={},start={},end={}".format(match_cel.group(i), match_cel.start(), match_cel.end()))
            cel=match_cel.group(i)
        except:
            cel = ""
        i+=1
    cels.append(cel)
    dnis.append(dni)
    # print('{},{}'.format(dni,cel))

In [9]:
np_cels = np.array(cels)
np_dnis = np.array(dnis)
np_mails = np.array(mails)
np_lines = np.array(lines)

In [10]:
np_dnis[:3], np_cels[:3], azure_files[:3], np_mails[:3], np_lines

(array(['', '09900151', '45277490'], dtype='<U8'),
 array(['', '916881398', ''], dtype='<U9'),
 array(['D:\\GyS\\proyectos\\demoAzureVision\\cupones\\MarzoCompras-0303072019121014_00343.jpg',
        'D:\\GyS\\proyectos\\demoAzureVision\\cupones\\MarzoCompras-0303072019121017_00450.jpg',
        'D:\\GyS\\proyectos\\demoAzureVision\\cupones\\MarzoCompras-0303072019121018_00497.jpg'],
       dtype='<U84'),
 array(['', 'ims0663@yahooes', 'syay1526@hotmailcom'], dtype='<U33'),
 array(['Nombres y Apellidos: TERESA HURTADOCASTILLODNI:of yoly / Telefono: 536 765.Direccion:COMAAU . 22 DE AGOSTO) 265Distrito: Tere hurtado of intheE-mail:Centro Comercial:- COMPS',
        'Nombres y Apellidos:MariamillaChauca PalominoDN09900151Telefono:916881 398Direccion:JR. FELIPE ARIAS392Distrito:J. M . PE-mail:ims - 0663@ yahoo, esCentro Comercial:Plaza Norte',
        'Nombres y Apellidos: SHERESNARDA RSAACUNA YNGADNI:45 2 7-7 490Telefono: 7552) 1928Direccion:Jr. Garala zo de la lega# 414 - Sam AgustinDist

In [15]:
check_field = lambda a, b : b in a
v_check_field = np.vectorize(check_field)

In [22]:
np_lines[1]

'Nombres y Apellidos:MariamillaChauca PalominoDN09900151Telefono:916881 398Direccion:JR. FELIPE ARIAS392Distrito:J. M . PE-mail:ims - 0663@ yahoo, esCentro Comercial:Plaza Norte'

In [21]:
v_check_field(np_lines,'DNI:')

array([ True, False,  True, ...,  True, False,  True])

In [27]:
v_check_field(np_lines,'DNI').sum() / np_lines.size

0.9830028328611898

In [23]:
v_check_field(np_lines,'Telefono:').sum() / np_lines.size

0.8951841359773371

In [25]:
v_check_field(np_lines,'Telefono').sum() / np_lines.size

0.9896128423040604

In [13]:
np_dnis.size, np_cels.size, azure_files.size, np_mails.size

(1059, 1059, 1059, 1059)

In [14]:
# Getting np arrays from saved
ruta = 'D:\\GyS\\proyectos\\cuponesWong\\proceso_definitivo\\process\\muestra1060\\'

n_files = np.load(ruta + 'files.npy', allow_pickle=True)
n_dnis = np.load(ruta + 'dnis.npy', allow_pickle=True)
n_cels = np.load(ruta + 'cels.npy', allow_pickle=True)
n_mails = np.load(ruta + 'mails.npy', allow_pickle=True)

In [15]:
n_files.size, n_dnis.size, n_cels.size, n_mails.size

(1859, 1859, 1859, 1859)

In [16]:
azure_files[5].split('\\')[-1]

'MarzoCompras-0303072019121032_00998.jpg'

In [17]:
np.argwhere(n_files=='MarzoCompras-0303072019121032_00998.jpg')[0][0]

1034

In [18]:
def distance(str1, str2):
    d=dict()
    for i in range(len(str1)+1):
        d[i]=dict()
        d[i][0]=i
    for i in range(len(str2)+1):
        d[0][i] = i
    for i in range(1, len(str1)+1):
        for j in range(1, len(str2)+1):
            d[i][j] = min(d[i][j-1]+1, d[i-1][j]+1, d[i-1][j-1]+(not str1[i-1] == str2[j-1]))
    return d[len(str1)][len(str2)]

In [19]:
filenames=[]

est_cels = []
real_cels = []
lev_cels = []

est_dnis = []
real_dnis = []
lev_dnis = []

azure_mails = []
real_mails = []
lev_mails = []

for i in range(azure_files.size):
# for i in range(5):
    # print('{}'.format(np.argwhere(m_files==np_res_filenames[i]+'.jpg')))
    f = azure_files[i].split('\\')[-1]
    filenames.append(f)
    
    # Correo
    try:
        mail_estimated = np_mails[i]
    except:
        mail_estimated = ""
    try:
        imx = np.argwhere(n_files==f)[0][0]
        real_mail = n_mails[imx]
    except:
        real_mail = "NN"
    
    try:
        dm = distance(mail_estimated,real_mail)
    except:
        dm = 50
    
    # DNI
    try:
        dni_estimated = np_dnis[i]
    except:
        dni_estimated = ""
    try:
        ix = np.argwhere(n_files==f)[0][0]
        real_dni = n_dnis[ix]
    except:
        real_dni = "NN"
    
    try:
        dd = distance(dni_estimated,real_dni)
    except:
        dd = 8
    
    # Cel
    try:
        cel_estimated = np_cels[i]
    except:
        cel_estimated = ""
    try:
        ixc = np.argwhere(n_files==f)[0][0]
        real_cel = n_cels[ixc]
    except:
        real_cel = "NN"
    
    try:
        dc = distance(cel_estimated,real_cel)
    except:
        dc = 8
    
    est_cels.append(cel_estimated)
    real_cels.append(real_cel)
    lev_cels.append(dc)
    
    est_dnis.append(dni_estimated)
    real_dnis.append(real_dni)
    lev_dnis.append(dd)
    
    azure_mails.append(mail_estimated)
    real_mails.append(real_mail)
    lev_mails.append(dm)    
    # print('{},{},{},{}'.format(azure_files[i]+'.jpg',dni_estimated,real_dni,dd))
    # print('{},{},{},{}'.format(azure_files[i]+'.jpg',cel_estimated,real_cel,dc))
    # print('{},{},{},{}'.format(f,mail_estimated,real_mail,dm))

In [20]:
np_est_dnis = np.array(est_dnis)
np_real_dnis = np.array(real_dnis)
np_lev_dnis = np.array(lev_dnis)

np_est_cels = np.array(est_cels)
np_real_cels = np.array(real_cels)
np_lev_cels = np.array(lev_cels)

np_azure_mails = np.array(azure_mails)
np_real_mails = np.array(real_mails)
np_lev_mails = np.array(lev_mails)


In [21]:
np.unique(np_lev_dnis), np.unique(np_lev_cels), np.unique(np_lev_mails), np_lev_dnis.size, np_lev_cels.size, np_lev_mails.size

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]),
 1059,
 1059,
 1059)

In [22]:
(np_est_dnis == '').sum() / np_est_dnis.size

0.03588290840415486

In [23]:
(np_est_cels == '').sum() / np_est_cels.size

0.295561850802644

In [26]:
result_df = pd.DataFrame({'filename':filenames})

result_df['azure_dni'] = np_est_dnis
result_df['real_dni'] = np_real_dnis
result_df['lev_dni'] = np_lev_dnis

result_df['azure_tel'] = np_est_cels
result_df['real_tel'] = np_real_cels
result_df['lev_tel'] = np_lev_cels

result_df['azure_mail'] = np_azure_mails
result_df['real_mail'] = np_real_mails
result_df['lev_mail'] = np_lev_mails

In [31]:
d = ['MarzoCompras-0303072019164230_00719.jpg',
 'MarzoCompras-0303072019164231_00751.jpg',
 'MarzoCompras-0303072019164233_00817.jpg',
 'MarzoCompras-0303072019164234_00853.jpg',
 'MarzoCompras-0303072019164234_00886.jpg']

In [30]:
result_df.head()

,filename,azure_dni,real_dni,lev_dni,azure_tel,real_tel,lev_tel,azure_mail,real_mail,lev_mail
0,MarzoCompras-0303072019121014_00343.jpg,,09740147,8,,5367652,7,,,0
1,MarzoCompras-0303072019121017_00450.jpg,09900151,09900151,0,916881398,976881398,1,ims0663@yahooes,IMS. 0663@yahoo.es,6
2,MarzoCompras-0303072019121018_00497.jpg,45277490,45279490,1,,955211928,9,syay1526@hotmailcom,sgay 1526@hotmail.com,3
3,MarzoCompras-0303072019121019_00540.jpg,73856082,73856082,0,943033279,993033279,1,Marrelaodg46mallcon,Mariela0194@ Gmail.com,10
4,MarzoCompras-0303072019121019_00541.jpg,,08281361,8,,949357898,9,,,0


In [32]:
result_df[np.isin(result_df.filename,d)]

,filename,azure_dni,real_dni,lev_dni,azure_tel,real_tel,lev_tel,azure_mail,real_mail,lev_mail
29,MarzoCompras-0303072019164231_00751.jpg,10160763,10160763,0,987592321,987592321,0,PATINO75469mall,PATINOV 754 @ g mail,8
30,MarzoCompras-0303072019164233_00817.jpg,98158209,19448315,7,981582093,981582093,0,Solomonacce@yohoocom,soluciones acoe@yahoo .com,11


In [55]:
result_df.groupby(['lev_mail']).agg({'filename':'count'})

,filename
lev_mail,
0,399
1,42
2,45
3,53
4,66
5,63
6,67
7,58
8,46


In [49]:
result_df.sort_values('lev_mail',ascending=True).head(400)

,filename,azure_mail,real_mail,lev_mail
0,MarzoCompras-0303072019121014_00343.jpg,,,0
359,MarzoCompras_EL_B4_20190709095715_00187.jpg,,,0
700,MarzoCompras_MD_B2_20190704130454_00876.jpg,,,0
356,MarzoCompras_EL_B4_20190708180738_00978.jpg,,,0
353,MarzoCompras_EL_B4_20190708180723_00541.jpg,,,0
352,MarzoCompras_EL_B4_20190708180721_00501.jpg,,,0
351,MarzoCompras_EL_B4_20190708173013_00807.jpg,,,0
710,MarzoCompras_MD_B2_20190704145946_00311.jpg,,,0
349,MarzoCompras_EL_B4_20190708172955_00285.jpg,,,0
711,MarzoCompras_MD_B2_20190704145950_00491.jpg,,,0


In [80]:
result_df[result_df.lev_mail==3].sort_values('real_mail',ascending=False)

,filename,azure_mail,real_mail,lev_mail
40,MarzoCompras_0202072019182419_00232.jpg,zahing88@hotmailcom,zahina 88@hotmail.com,3
884,MarzoCompras_MD_B5_20190709181758_00529.jpg,Yvoners@hotmailcom,yvonecs@hotmail.com,3
819,MarzoCompras_MD_B4_20190708153611_00093.jpg,Illants93@gmailcom,vllanto93@gmail.com,3
562,MarzoCompras_LV_B2_20190705103303_00430.jpg,villamari1206@imalcom,villamari1206@gmail.com,3
211,MarzoCompras_EL_B2_20190704114142_00928.jpg,victoriatapirhotmailcon,victoriatapi@hotmail.com,3
851,MarzoCompras_MD_B5_20190709155045_00146.jpg,seritalovehotmailcom,veritalove@hotmail.com,3
784,MarzoCompras_MD_B2_20190705134853_00903.jpg,"torvic_Peru""hotmailcom",torvic_Peu@hotmail.com,3
792,MarzoCompras_MD_B3_20190705143124_00350.jpg,thejuli98@hotmailcom,the-juli-98@hotmail.com,3
719,MarzoCompras_MD_B2_20190704154634_00762.jpg,sheshi_reinosohotmailco,sheshi_reinoso@hotmail.com,3
2,MarzoCompras-0303072019121018_00497.jpg,syay1526@hotmailcom,sgay 1526@hotmail.com,3


In [76]:
result_df[result_df.lev_mail<=4].count() , result_df.count(), result_df[result_df.lev_mail==0].count()

(filename      605
 azure_mail    605
 real_mail     605
 lev_mail      605
 dtype: int64, filename      1059
 azure_mail    1059
 real_mail     1059
 lev_mail      1059
 dtype: int64, filename      399
 azure_mail    399
 real_mail     399
 lev_mail      399
 dtype: int64)

In [77]:
605/1059

0.5712936732766761

In [78]:
(605-399)/1059

0.19452313503305005

In [79]:
398/1059

0.37582625118035884

# Resultados tablas

In [26]:
data = []
indexes = []
columns = ['dni','cel']
for i in range(0,12):
    indexes.append(i)
    data.append([(np_lev_dnis==i).sum(),(np_lev_cels==i).sum()])
    # print('{}'.format(i))

In [27]:
import pandas as pd
res_df = pd.DataFrame(data=data,# values
             index=indexes,# 1st column as index
             columns=columns)
res_df['dni_percent'] = (res_df['dni']/res_df['dni'].sum())*100
res_df['cel_percent'] = (res_df['cel']/res_df['cel'].sum())*100
res_df['status'] = np.where(res_df.index<=1, 'aceptado', 'no aceptado')

In [28]:
summ_df = res_df.groupby(['status'], as_index=True).agg({'dni': 'sum','cel':'sum','dni_percent':'sum','cel_percent':'sum'})

In [29]:
summ_df

,dni,cel,dni_percent,cel_percent
status,,,,
aceptado,788,588,74.409821,55.524079
no aceptado,271,471,25.590179,44.475921


In [30]:
res_df

,dni,cel,dni_percent,cel_percent,status
0,581,415,54.863078,39.187913,aceptado
1,207,173,19.546742,16.336166,aceptado
2,79,63,7.459868,5.949008,no aceptado
3,23,22,2.171860,2.077432,no aceptado
4,3,7,0.283286,0.661001,no aceptado
5,8,8,0.755430,0.755430,no aceptado
6,34,6,3.210576,0.566572,no aceptado
7,44,123,4.154863,11.614731,no aceptado
8,78,74,7.365439,6.987724,no aceptado
9,1,157,0.094429,14.825307,no aceptado


# Azure menos reconocidos localmente

In [171]:
# Loading model ok local files
ruta = 'C:\\git\\yolov3\\keras-yolo3\\muestra_ok\\'
ok_filepaths = np.load(ruta+'muestra_ok_local_20190804.npy')

In [172]:
ok_filepaths[:4]

array([['MarzoCompras_0202072019164515_01004'],
       ['MarzoCompras_0203072019122714_00118'],
       ['MarzoCompras_0203072019122717_00226'],
       ['MarzoCompras_0203072019122720_00366']], dtype='<U39')

In [173]:
ok = []
for i in range(ok_filepaths.size):
    n = ok_filepaths[i][0]+'.jpg'
    ok.append(n)
    # print('{}'.format(n))
ok_files = np.array(ok)

In [174]:
azure_filenames = []
for i in range(azure_files.size):
    f = azure_files[i].split('\\')[-1]
    azure_filenames.append(f)

In [175]:
azure_filenames = np.array(azure_filenames)

In [176]:
azure_files.size, np_lev_cels.size, np_lev_dnis.size, azure_filenames.size

(1059, 1059, 1059, 1059)

In [177]:
np.argwhere(azure_filenames=='MarzoCompras_MD_B5_20190709181758_00529.jpg')[0][0]

884

In [178]:
idxs = []
for i in range(ok_files.size):
    idx = np.argwhere(azure_filenames==ok_files[i])[0][0]
    # print('{}'.format(idx))
    idxs.append(idx)

In [179]:
dif_dnis = np.delete(np_lev_dnis, idxs)
dif_cels = np.delete(np_lev_cels, idxs)

In [186]:
np.unique(dif_dnis), np.unique(dif_cels), dif_dnis.size, dif_cels.size

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 792,
 792)

In [181]:
data_dif = []
indexes_dif = []
columns = ['dni','cel']
for i in range(0,12):
    indexes_dif.append(i)
    data_dif.append([(dif_dnis==i).sum(),(dif_cels==i).sum()])
    # print('{}'.format(i))

In [182]:
import pandas as pd
dif_df = pd.DataFrame(data=data_dif,# values
             index=indexes_dif,# 1st column as index
             columns=columns)
dif_df['dni_percent'] = (dif_df['dni']/dif_df['dni'].sum())*100
dif_df['cel_percent'] = (dif_df['cel']/dif_df['cel'].sum())*100
dif_df['status'] = np.where(dif_df.index <= 1, 'aceptado', 'no aceptado')

In [210]:
dif_df

,dni,cel,dni_percent,cel_percent,status
0,395,295,49.873737,37.247475,aceptado
1,165,122,20.833333,15.404040,aceptado
2,67,48,8.459596,6.060606,no aceptado
3,20,17,2.525253,2.146465,no aceptado
4,3,4,0.378788,0.505051,no aceptado
5,7,6,0.883838,0.757576,no aceptado
6,29,6,3.661616,0.757576,no aceptado
7,38,102,4.797980,12.878788,no aceptado
8,66,56,8.333333,7.070707,no aceptado
9,1,129,0.126263,16.287879,no aceptado


In [214]:
dif_summ_df = dif_df.groupby(['status'], as_index=True).agg({'dni': 'sum','cel':'sum','dni_percent':'sum','cel_percent':'sum'})
dif_summ_df['conjunto'] = (dif_summ_df['dni_percent'] + dif_summ_df['cel_percent']) / 2

In [215]:
dif_summ_df

,dni,cel,dni_percent,cel_percent,conjunto
status,,,,,
aceptado,560,417,70.707071,52.651515,61.679293
no aceptado,232,375,29.292929,47.348485,38.320707


In [211]:
dif_ajustada = dif_summ_df.iloc[:,:2]
dif_ajustada['dni_percent_total_muestra'] = (dif_ajustada['dni'] / 1059)*100
dif_ajustada['cel_percent_total_muestra'] = (dif_ajustada['cel'] / 1059)*100
dif_ajustada['conjunto'] = (dif_ajustada['dni_percent_total_muestra'] + dif_ajustada['cel_percent_total_muestra'])/2

In [213]:
dif_ajustada

,dni,cel,dni_percent_total_muestra,cel_percent_total_muestra,conjunto
status,,,,,
aceptado,560,417,52.880076,39.376771,46.128423
no aceptado,232,375,21.907460,35.410765,28.659112
